In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import os, gc, random

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

pd.set_option('display.max_columns', 140) #最大表示列数の指定


# fix seed
seed = 2021
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

## Dataset Class

In [ ]:
from torch.utils.data import Dataset
from torch import nn
 
class JSMP_Dataset(Dataset):
     
    def __init__(self, file_path, window_size):
        # valiables
        self.file_path = file_path
        self.window_size = window_size
        
        # read csv
        train = pd.read_csv(file_path)
        
        # pre processing
        train = train.query('date > 85').reset_index(drop = True) 
        #train = train[train['weight'] != 0]
        train.fillna(train.mean(),inplace=True)
        train['action'] = ((train['resp'].values) > 0).astype(int)
        
        resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
        self.features = [c for c in train.columns if "feature" in c]
        self.f_mean = np.mean(train[self.features[1:]].values,axis=0)
        
        self.X_train = train.loc[:, train.columns.str.contains('feature')].values
        self.y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T
        
        self.X_train = torch.from_numpy(self.X_train).float()
        self.y_train = torch.from_numpy(self.y_train).float()
        
        # reduce memory
        del train
        gc.collect()
 
    def __len__(self):
        return len(self.X_train) - self.window_size
     
    def __getitem__(self, i):
        data = self.X_train[i:(i+ self.window_size), :] 
        label = self.y_train[i + self.window_size - 1]
 
        return data, label

In [ ]:
window_size = 5
file_path = '/kaggle/input/jane-street-market-prediction/train.csv'
ds = JSMP_Dataset(file_path, window_size)

## Network(TCN)
https://github.com/locuslab/TCN/blob/master/TCN/tcn.py

In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.fc = nn.Linear(130 * num_channels[-1], output_size)

    def forward(self, inputs):
        """Inputs have to have dimension (N, C_in, L_in)"""
        y1 = self.tcn(inputs)  # input should have dimension (N, C, L)
        #y1 = torch.flatten(y1, start_dim=1)
        #o = self.fc(y1)
        return y1#torch.sigmoid(o)

In [ ]:
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        
        self.fc1 = nn.Linear(130 * num_channels[-1], 128)
        self.dropout1 = nn.Dropout(dropout)
        self.batch_norm1 = nn.BatchNorm1d(128)
        self.LeakyReLU1 = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        
        self.fc2 = nn.Linear(128, 128)
        self.dropout2 = nn.Dropout(dropout)
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.LeakyReLU2 = nn.LeakyReLU(negative_slope=0.01, inplace=True)
        
        self.fc3 = nn.Linear(128, output_size)
        
    def forward(self, inputs):
        """Inputs have to have dimension (N, C_in, L_in)"""
        y1 = self.tcn(inputs)  # input should have dimension (N, C, L)
        y1 = torch.flatten(y1, start_dim=1)
        
        y1 = self.fc1(y1)
        y1 = self.batch_norm1(y1)
        y1 = self.LeakyReLU1(y1)
        y1 = self.dropout1(y1)
        
        y1 = self.fc2(y1)
        y1 = self.batch_norm2(y1)
        y1 = self.LeakyReLU2(y1)
        y1 = self.dropout2(y1)
        
        o = self.fc3(y1)
        return torch.sigmoid(o)

## Inference

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('use devise:', device)

# load model
net1 = TCN(input_size=5, output_size=5, num_channels=[16, 8, 4, 2], kernel_size=2, dropout=0.5)
net2 = TCN(input_size=5, output_size=5, num_channels=[16, 8, 4, 2], kernel_size=2, dropout=0.5)

net1.load_state_dict(torch.load('/kaggle/input/jsmp-tcn-pytorch/best_accuracy_model.mdl', map_location=torch.device(device)))
net2.load_state_dict(torch.load('/kaggle/input/jsmp-tcn-pytorch/best_loss_model.mdl', map_location=torch.device(device)))

net1.eval()
net2.eval()

In [ ]:
th = 0.5

import janestreet
env = janestreet.make_env()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('use devise:', device)

for i, (test_df, pred_df) in enumerate(env.iter_test()):
    x_tt = test_df.loc[:, ds.features].values
    if np.isnan(x_tt[:, 1:].sum()):
        x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * ds.f_mean
    
    # make window data
    if i == 0:
        x_window = x_tt.copy()
    elif i < window_size: 
        x_window = np.concatenate([x_window, x_tt], axis=0)
    else:
        x_window = np.concatenate([x_window[1:, :], x_tt], axis=0)
    
    if i < window_size - 1:
        # pass 
        pred_df.action = 0
    else:
        # prediction
        if test_df['weight'].item() > 0:
            inputs = torch.Tensor(x_window).unsqueeze(0).to(device)
            outputs = (net1(inputs) + net2(inputs)) / 2
            pred = (torch.median(outputs, axis=1).values > th).long()
            pred_df.action = pred.item()
            #print(pred.item())
        else:
            pred_df.action = 0
        
    env.predict(pred_df)